# Data Combine

In [1]:
import numpy as np
import pandas as pd
import pylab as pl
import geopandas as gpd
import datetime
from matplotlib import gridspec
from fiona.crs import from_epsg
import shapely

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_row', 500)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


read all data files

In [2]:
taxi = pd.read_csv('lga_taxi_clean.csv')

In [3]:
taxi.shape

(813512, 24)

In [4]:
taxi.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,trip_time,pickup_month,pickup_day,pickup_hour,pickup_minute,day_id,hour_id,location_day_id,yesterday_avg_trip_time,location_yesterday_id,location_hour_id,perious_hour_avg_trip_time,two_hour_ago_avg_trip_time,week,if_Friday,if_weekend,tpep_pickup_date,is_hoilday,is_before_hoilday,is_after_hoilday
0,2018-01-02 04:49:00,2018-01-02 05:13:00,11.52,100,24,1,2,4,49,1,28,"(100L, 1L)",23.121212,"(100L, 1L)","(100L, 28L)",21.0,17.0,1,0,0,2018-01-02,0,0,1
1,2018-01-02 04:41:00,2018-01-02 05:02:00,8.99,170,21,1,2,4,41,1,28,"(170L, 1L)",19.730159,"(170L, 1L)","(170L, 28L)",20.0,17.0,1,0,0,2018-01-02,0,0,1
2,2018-01-02 04:38:00,2018-01-02 04:53:00,8.67,170,15,1,2,4,38,1,28,"(170L, 1L)",19.730159,"(170L, 1L)","(170L, 28L)",20.0,17.0,1,0,0,2018-01-02,0,0,1
3,2018-01-02 04:15:00,2018-01-02 04:40:00,12.20,230,25,1,2,4,15,1,28,"(230L, 1L)",25.372822,"(230L, 1L)","(230L, 28L)",20.5,17.0,1,0,0,2018-01-02,0,0,1
4,2018-01-02 04:39:00,2018-01-02 05:02:00,7.99,48,23,1,2,4,39,1,28,"(48L, 1L)",26.047619,"(48L, 1L)","(48L, 28L)",21.0,23.0,1,0,0,2018-01-02,0,0,1


In [5]:
weather = pd.read_csv('clean_weather_with_abnormal.csv')

In [7]:
weather = weather[['HourlyWetBulbTemperature', 'HourlyRelativeHumidity', 'HourlyStationPressure', 'HourlyVisibility', 'HourlyWindSpeed', 'hour_id']]

In [8]:
weather.shape

(8760, 6)

In [9]:
weather.head()

,HourlyWetBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed,hour_id
0,7.0,55.0,30.16,10.00,9.0,0
1,7.0,55.0,30.17,10.00,10.0,1
2,6.0,57.0,30.18,10.00,10.0,2
3,6.0,55.0,30.18,10.00,7.0,3
4,6.0,55.0,30.20,10.00,9.0,4


In [10]:
accident_day = pd.read_csv('accident_day_clean.csv')

In [11]:
accident_day.shape

(213787, 3)

In [12]:
accident_day.head()

,LocationID,date,day_id
0,2,2018-08-25,235
1,2,2018-05-04,122
2,2,2018-05-22,140
3,2,2018-05-11,129
4,2,2018-08-04,214


In [13]:
accident_day = accident_day[['LocationID', 'day_id']]
accident_day['is_accident_today'] = 1

In [14]:
accident_day.head()

,LocationID,day_id,is_accident_today
0,2,235,1
1,2,122,1
2,2,140,1
3,2,129,1
4,2,214,1


In [15]:
accident_count = pd.read_csv('accident_count_clean.csv')

In [16]:
accident_count.shape

(258, 2)

In [17]:
accident_count.head()

,LocationID,Counts
0,2,8
1,3,598
2,4,273
3,5,189
4,6,439


filter taxi columns

In [18]:
taxi.columns

Index([u'tpep_pickup_datetime', u'tpep_dropoff_datetime', u'trip_distance',
       u'PULocationID', u'trip_time', u'pickup_month', u'pickup_day',
       u'pickup_hour', u'pickup_minute', u'day_id', u'hour_id',
       u'location_day_id', u'yesterday_avg_trip_time',
       u'location_yesterday_id', u'location_hour_id',
       u'perious_hour_avg_trip_time', u'two_hour_ago_avg_trip_time', u'week',
       u'if_Friday', u'if_weekend', u'tpep_pickup_date', u'is_hoilday',
       u'is_before_hoilday', u'is_after_hoilday'],
      dtype='object')

In [19]:
taxi_clean = taxi[['trip_time', 'trip_distance', 'PULocationID', 'pickup_month', 'pickup_day', 'pickup_hour', 'pickup_minute', 
                   'yesterday_avg_trip_time', 'perious_hour_avg_trip_time', 'two_hour_ago_avg_trip_time', 'if_Friday', 'if_weekend', 
                   'is_hoilday', 'is_before_hoilday', 'is_after_hoilday', 'day_id', 'hour_id']]

taxi with accident count

In [20]:
taxi_clean = taxi_clean.merge(accident_count, left_on = 'PULocationID', right_on = 'LocationID', how = 'left').drop(columns = ['LocationID'], axis = 1)

In [21]:
taxi_clean.head()

,trip_time,trip_distance,PULocationID,pickup_month,pickup_day,pickup_hour,pickup_minute,yesterday_avg_trip_time,perious_hour_avg_trip_time,two_hour_ago_avg_trip_time,if_Friday,if_weekend,is_hoilday,is_before_hoilday,is_after_hoilday,day_id,hour_id,Counts
0,24,11.52,100,1,2,4,49,23.121212,21.0,17.0,0,0,0,0,1,1,28,609.0
1,21,8.99,170,1,2,4,41,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0
2,15,8.67,170,1,2,4,38,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0
3,25,12.20,230,1,2,4,15,25.372822,20.5,17.0,0,0,0,0,1,1,28,822.0
4,23,7.99,48,1,2,4,39,26.047619,21.0,23.0,0,0,0,0,1,1,28,1393.0


In [22]:
sum(taxi_clean.isna())

trip_time                       0
trip_distance                   0
PULocationID                    0
pickup_month                    0
pickup_day                      0
pickup_hour                     0
pickup_minute                   0
yesterday_avg_trip_time         0
perious_hour_avg_trip_time      0
two_hour_ago_avg_trip_time      0
if_Friday                       0
if_weekend                      0
is_hoilday                      0
is_before_hoilday               0
is_after_hoilday                0
day_id                          0
hour_id                         0
Counts                        434
dtype: int64

In [23]:
taxi_clean = taxi_clean.fillna(0)

In [24]:
sum(taxi_clean.isna())

trip_time                     0
trip_distance                 0
PULocationID                  0
pickup_month                  0
pickup_day                    0
pickup_hour                   0
pickup_minute                 0
yesterday_avg_trip_time       0
perious_hour_avg_trip_time    0
two_hour_ago_avg_trip_time    0
if_Friday                     0
if_weekend                    0
is_hoilday                    0
is_before_hoilday             0
is_after_hoilday              0
day_id                        0
hour_id                       0
Counts                        0
dtype: int64

taxi with accident location daily

In [25]:
taxi_clean = taxi_clean.merge(accident_day, left_on = ['PULocationID', 'day_id'], right_on = ['LocationID', 'day_id'], how = 'left').drop(columns = ['LocationID'], axis = 1)

In [26]:
sum(taxi_clean.isna())

trip_time                          0
trip_distance                      0
PULocationID                       0
pickup_month                       0
pickup_day                         0
pickup_hour                        0
pickup_minute                      0
yesterday_avg_trip_time            0
perious_hour_avg_trip_time         0
two_hour_ago_avg_trip_time         0
if_Friday                          0
if_weekend                         0
is_hoilday                         0
is_before_hoilday                  0
is_after_hoilday                   0
day_id                             0
hour_id                            0
Counts                             0
is_accident_today             142839
dtype: int64

In [27]:
taxi_clean = taxi_clean.fillna(0)

In [28]:
sum(taxi_clean.isna())

trip_time                     0
trip_distance                 0
PULocationID                  0
pickup_month                  0
pickup_day                    0
pickup_hour                   0
pickup_minute                 0
yesterday_avg_trip_time       0
perious_hour_avg_trip_time    0
two_hour_ago_avg_trip_time    0
if_Friday                     0
if_weekend                    0
is_hoilday                    0
is_before_hoilday             0
is_after_hoilday              0
day_id                        0
hour_id                       0
Counts                        0
is_accident_today             0
dtype: int64

In [29]:
taxi_clean.head()

,trip_time,trip_distance,PULocationID,pickup_month,pickup_day,pickup_hour,pickup_minute,yesterday_avg_trip_time,perious_hour_avg_trip_time,two_hour_ago_avg_trip_time,if_Friday,if_weekend,is_hoilday,is_before_hoilday,is_after_hoilday,day_id,hour_id,Counts,is_accident_today
0,24,11.52,100,1,2,4,49,23.121212,21.0,17.0,0,0,0,0,1,1,28,609.0,0.0
1,21,8.99,170,1,2,4,41,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0,1.0
2,21,8.99,170,1,2,4,41,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0,1.0
3,21,8.99,170,1,2,4,41,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0,1.0
4,15,8.67,170,1,2,4,38,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0,1.0


taxi with weather

In [30]:
taxi_clean = taxi_clean.merge(weather, left_on = 'hour_id', right_on = 'hour_id', how = 'left')
#.drop(columns = ['LocationID'], axis = 1)

In [31]:
taxi_clean.head()

,trip_time,trip_distance,PULocationID,pickup_month,pickup_day,pickup_hour,pickup_minute,yesterday_avg_trip_time,perious_hour_avg_trip_time,two_hour_ago_avg_trip_time,if_Friday,if_weekend,is_hoilday,is_before_hoilday,is_after_hoilday,day_id,hour_id,Counts,is_accident_today,HourlyWetBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed
0,24,11.52,100,1,2,4,49,23.121212,21.0,17.0,0,0,0,0,1,1,28,609.0,0.0,10.0,59.0,30.22,10.00,5.0
1,21,8.99,170,1,2,4,41,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0,1.0,10.0,59.0,30.22,10.00,5.0
2,21,8.99,170,1,2,4,41,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0,1.0,10.0,59.0,30.22,10.00,5.0
3,21,8.99,170,1,2,4,41,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0,1.0,10.0,59.0,30.22,10.00,5.0
4,15,8.67,170,1,2,4,38,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0,1.0,10.0,59.0,30.22,10.00,5.0


In [40]:
taxi_clean.to_csv('clean_data_with_feature.csv', index = False)

Clean the feature data which cannot convert to float.

In [41]:
raw_data = pd.read_csv("clean_data_with_feature.csv")

In [42]:
raw_data.head()

,trip_time,trip_distance,PULocationID,pickup_month,pickup_day,pickup_hour,pickup_minute,yesterday_avg_trip_time,perious_hour_avg_trip_time,two_hour_ago_avg_trip_time,if_Friday,if_weekend,is_hoilday,is_before_hoilday,is_after_hoilday,day_id,hour_id,Counts,is_accident_today,HourlyWetBulbTemperature,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWindSpeed
0,24,11.52,100,1,2,4,49,23.121212,21.0,17.0,0,0,0,0,1,1,28,609.0,0.0,10.0,59.0,30.22,10,5.0
1,21,8.99,170,1,2,4,41,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0,1.0,10.0,59.0,30.22,10,5.0
2,21,8.99,170,1,2,4,41,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0,1.0,10.0,59.0,30.22,10,5.0
3,21,8.99,170,1,2,4,41,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0,1.0,10.0,59.0,30.22,10,5.0
4,15,8.67,170,1,2,4,38,19.730159,20.0,17.0,0,0,0,0,1,1,28,1287.0,1.0,10.0,59.0,30.22,10,5.0


In [45]:
for item in raw_data['HourlyStationPressure']:
    try:
        float(item)
    except ValueError:
        #print(item.split('s'))
        print(item)
        break

29.29s


In [50]:
def remove_s(x):
    try:
        x = float(x)
        return(x)
    except ValueError:
        no_s = float(x.split('s')[0])
        return(no_s)

#raw_data['HourlyWetBulbTemperature'] = raw_data['HourlyWetBulbTemperature'].apply(remove_s).tolist()

In [52]:
#set(raw_data['HourlyStationPressure'].apply(remove_s).tolist())

In [53]:
raw_data['HourlyStationPressure'] = raw_data['HourlyStationPressure'].apply(remove_s).tolist()

In [54]:
def remove_V(x):
    try:
        return(float(x))
    except ValueError:
        no_s = float(x.split('V')[0])
        return(no_s)

raw_data['HourlyVisibility'] = raw_data['HourlyVisibility'].apply(remove_V).tolist()

In [55]:
raw_data.to_csv('clean_data_with_feature.csv', index = False)